# Coronavirus Data Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
spark.sparkContext.applicationId

'application_1636170550178_0352'

In [3]:
pd.options.display.max_columns = 35

In [4]:
from foresight.discern import broadcast_discern, push_discern, pop_discern

### care management new default will be 5e259fd5-75b5-4d49-82d3-2d4e92dca831 for any new clients or 
###  any algorithm wiki not line for line with Cerner Standard then would be using the new context  

pd.set_option('display.max_colwidth', None)

# Location of the discernontology database
root = 's3://consult-datalab-persistence-s3-data/discernontology/v1/'   ##<------------- change it to your stack without the v1/

contextid = ['da0acee6-1e2e-4384-9fc7-a58c259d0c50', '5E259FD575B54D4982D32D4E92DCA831'] 

# Loop that removes all hypthens and captializes all letters. Sets up the file location
for contextid in contextid:
    contextid = contextid.upper().replace("-", "")
    push_discern(spark, contextid, discern_root=root)
    print(contextid)

defaultcontext = 'DA0ACEE61E2E43849FC7A58C259D0C50'   
defaultcontext1 = '5E259FD575B54D4982D32D4E92DCA831' 

DA0ACEE61E2E43849FC7A58C259D0C50
5E259FD575B54D4982D32D4E92DCA831


In [5]:
db = "real_world_data_2021_Q2"
spark.sql("USE {}".format(db))
print("Using database: {},".format(db))

Using database: real_world_data_2021_Q2,


In [6]:
spark.sql('show tables').toPandas()

,database,tableName,isTemporary
0,real_world_data_2021_q2,allergy,False
1,real_world_data_2021_q2,clinical_event,False
2,real_world_data_2021_q2,condition,False
3,real_world_data_2021_q2,demographics,False
4,real_world_data_2021_q2,encounter,False
5,real_world_data_2021_q2,immunization,False
6,real_world_data_2021_q2,lab,False
7,real_world_data_2021_q2,measurement,False
8,real_world_data_2021_q2,medication,False
9,real_world_data_2021_q2,medication_administration,False


### Create Variables for Queries

In [7]:
# Look back days variables
daysInterval = 365
print(daysInterval)
maxDays = 365
print(maxDays)
oneYearInterval=365
print(oneYearInterval)

# Username variable
user_id = 'rc047407'           ####<-------------------- change to your user id (Ex: tl068507)
userName = 'user_' + user_id
print(userName)

# Algorithm name variable
algorithmName='CoronavirusLengthOfStay_HighLevel'
print(algorithmName)

365
365
365
user_rc047407
CoronavirusLengthOfStay_HighLevel


In [8]:
spark.sql("""CREATE SCHEMA IF NOT EXISTS {userName}""".format(**{
    'userName':userName
})).show()

++
||
++
++



In [9]:
spark.sql(f"""select * FROM {userName}.{algorithmName}_Hospital h  """).printSchema()

root
 |-- tenant: integer (nullable = true)
 |-- tenant_bed_size: string (nullable = true)
 |-- tenant_speciality: string (nullable = true)
 |-- tenant_segment: string (nullable = true)
 |-- tenant_zip_code: string (nullable = true)
 |-- personId: string (nullable = true)
 |-- encounterId: string (nullable = true)
 |-- serviceDate: string (nullable = true)
 |-- dischargeDate: string (nullable = true)
 |-- length_of_stay: integer (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- deceased: integer (nullable = true)
 |-- discharged_to_skilled_nursing_facility: integer (nullable = true)
 |-- discharged_to_hospice: integer (nullable = true)
 |-- died_in_the_hospital: integer (nullable = true)
 |-- discharged_to_rehab_facility: integer (nullable = true)
 |-- discharged_to_long_term_care_facility: integer (nullable = true)
 |-- intensive_care_visit: integer (nullable = true)
 |-- intensive_care_medical_service: integer (nullable = true)
 |-- palliative_care_visit: integer (nullab

In [10]:
spark.sql("""select * from condition """).printSchema()

root
 |-- conditionid: string (nullable = true)
 |-- personid: string (nullable = true)
 |-- encounterid: string (nullable = true)
 |-- conditioncode: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- codingSystemId: string (nullable = true)
 |    |    |    |-- primaryDisplay: string (nullable = true)
 |-- effectivedate: string (nullable = true)
 |-- asserteddate: string (nullable = true)
 |-- type: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodi

In [11]:
conditionDetailDf = spark.sql(f"""


    SELECT distinct 
        h.tenant
        ,h.personId
        ,h.encounterId
        ,c.conditionId
        ,c.effectiveDate
        ,c.conditionCode.standard.id
        ,c.conditionCode.standard.primaryDisplay
        ,c.conditionCode.standard.codingSystemId
        ,c.type.standard.id
        ,c.type.standard.primaryDisplay
        ,c.type.standard.codingSystemId
        ,c.classification.standard.id
        ,c.classification.standard.primaryDisplay
        ,c.classification.standard.codingSystemId



  
        FROM {userName}.{algorithmName}_Hospital h  

       

        JOIN {db}.condition c
            ON  h.personId = c.personId
                and h.tenant = c.tenant
                and h.encounterId = c.encounterId
                AND  NOT ( HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CORONAVIRUS_COVID_19_DISEASE_CLIN',
                                        'CORONAVIRUS_COVID_19_POSITIVE_CLIN'), '{defaultcontext}' ) )
                AND NOT HAS_ANY_CONCEPT_IN_CONTEXT(c.status, array('ENTERED_IN_ERROR_PROBSTAT', 'ERROR_ENTRY_DELETED_QUAL', 
                                                       'CANCELED_PROBSTAT', 'CANCELED_QUAL', 'DEFERRED_QUAL',
                                                       'UNAUTHORIZED_QUAL', 'VOIDED_QUAL', 'REJECTED_QUAL'), '5E259FD575B54D4982D32D4E92DCA831')  
                
                            
      limit(25)
                       
   
   
""")

In [12]:
conditionDetailDf.toPandas()

,tenant,personId,encounterId,conditionId,effectiveDate,id,primaryDisplay,codingSystemId,id,primaryDisplay,codingSystemId,id,primaryDisplay,codingSystemId
0,68,30874fe9-182c-4485-8305-1ec7b0b4fc76,3c48e475-f020-4e0f-a837-a66b28beca94,8bcdd5ab-4881-4106-936b-c295101895a2,,Z79.4,Long term (current) use of insulin,2.16.840.1.113883.6.90,282291009,Diagnosis interpretation,2.16.840.1.113883.6.96,89100005,Final diagnosis (discharge),2.16.840.1.113883.6.96
1,68,30874fe9-182c-4485-8305-1ec7b0b4fc76,f88dd336-b555-4055-a67e-57f61b474e58,cd692962-4ab4-4ed6-b3de-3bd952e1508e,,E11.22,Type 2 diabetes mellitus with diabetic chronic kidney disease,2.16.840.1.113883.6.90,282291009,Diagnosis interpretation,2.16.840.1.113883.6.96,89100005,Final diagnosis (discharge),2.16.840.1.113883.6.96
2,68,7e6292ca-6cde-4f1e-a847-c9f6187f2346,43e31f5c-7512-4903-b376-6f1d8f69f4f8,75c99160-353b-4eb1-8cb4-413d53d17540,2020-05-13T19:00:00+00:00,None,None,None,282291009,Diagnosis interpretation,2.16.840.1.113883.6.96,406524005,Reason for visit diagnosis,2.16.840.1.113883.6.96
3,68,71a4a533-9996-4d72-9cf0-aed97158287e,97398957-b110-4eba-b292-faa2c84e1956,5426de49-640f-4a16-b94d-27b3f90971f1,,R29.810,Facial weakness,2.16.840.1.113883.6.90,282291009,Diagnosis interpretation,2.16.840.1.113883.6.96,89100005,Final diagnosis (discharge),2.16.840.1.113883.6.96
4,68,f1fdd2ee-0124-457d-8a29-ff02ece967de,9aea5827-6d36-43fc-9fd5-92299dea4ad0,62d94623-7fd1-4226-9271-396ed0ce98a5,,M54.5,Low back pain,2.16.840.1.113883.6.90,282291009,Diagnosis interpretation,2.16.840.1.113883.6.96,89100005,Final diagnosis (discharge),2.16.840.1.113883.6.96
5,68,05b1f5f5-26bc-44aa-a317-fc174590a097,4c94bd47-f6a3-4b0e-8b51-70f76d90cdc3,32364d43-de1c-41de-8a58-5f9b28e6457c,,R55,Syncope and collapse,2.16.840.1.113883.6.90,282291009,Diagnosis interpretation,2.16.840.1.113883.6.96,89100005,Final diagnosis (discharge),2.16.840.1.113883.6.96
6,68,a3270ba2-e703-428c-8264-4fb97698bc7f,12cba894-3274-4666-ad6b-247b4a311281,36708fd5-c685-4c17-b85a-162d3be2835c,,Z87.891,Personal history of nicotine dependence,2.16.840.1.113883.6.90,282291009,Diagnosis interpretation,2.16.840.1.113883.6.96,89100005,Final diagnosis (discharge),2.16.840.1.113883.6.96
7,68,b7d3b3b1-aa3e-4ca4-9740-a33fd1f1208e,1220823b-98e8-49b5-84f0-19a31a143402,6d334bee-c55f-477a-b480-cc708f774118,,R62.7,Adult failure to thrive,2.16.840.1.113883.6.90,282291009,Diagnosis interpretation,2.16.840.1.113883.6.96,89100005,Final diagnosis (discharge),2.16.840.1.113883.6.96
8,68,ceb8e805-039e-426c-b345-9fbd4cb3d0de,2f850a58-2007-4dd5-a74f-f2863827ac9e,91781220-655d-4399-b3ff-719ef779dc5c,,E10.22,Type 1 diabetes mellitus with diabetic chronic kidney disease,2.16.840.1.113883.6.90,282291009,Diagnosis interpretation,2.16.840.1.113883.6.96,89100005,Final diagnosis (discharge),2.16.840.1.113883.6.96
9,68,d055491f-8075-491a-8c26-a742fb943c79,aac94176-b75d-428c-9e9e-5937fd29e45b,d21e92f9-18c5-4944-848d-d0dcd588fcc2,,Z37.0,Single live birth,2.16.840.1.113883.6.90,282291009,Diagnosis interpretation,2.16.840.1.113883.6.96,89100005,Final diagnosis (discharge),2.16.840.1.113883.6.96


In [15]:
spark.sql(f"""select count(distinct personId)
    FROM {userName}.{algorithmName}_Hospital h  """).show()

+------------------------+
|count(DISTINCT personId)|
+------------------------+
|                  177699|
+------------------------+



In [11]:
conditionSummaryDf = spark.sql(f"""


    SELECT 
          c.conditionCode.standard.id
          ,c.conditionCode.standard.primaryDisplay as conditionCode
--        ,c.type.standard.primaryDisplay as conditionType
--        ,c.classification.standard.primaryDisplay as conditionClassification
        ,count(distinct  h.personId) as count

  
        FROM {userName}.{algorithmName}_Hospital h  

       

        JOIN {db}.condition c
            ON  h.personId = c.personId
                and h.tenant = c.tenant
                and h.encounterId = c.encounterId
                AND  NOT ( HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CORONAVIRUS_COVID_19_DISEASE_CLIN',
                                        'CORONAVIRUS_COVID_19_POSITIVE_CLIN'), '{defaultcontext}' ) )
                AND NOT HAS_ANY_CONCEPT_IN_CONTEXT(c.status, array('ENTERED_IN_ERROR_PROBSTAT', 'ERROR_ENTRY_DELETED_QUAL', 
                                                       'CANCELED_PROBSTAT', 'CANCELED_QUAL', 'DEFERRED_QUAL',
                                                       'UNAUTHORIZED_QUAL', 'VOIDED_QUAL', 'REJECTED_QUAL'), '5E259FD575B54D4982D32D4E92DCA831')  
              --  AND c.conditionCode.standard.primaryDisplay IS NOT NULL
                AND c.conditionCode.standard.id IS NOT NULL
                
    group by 1,2
    
    having count >= 1500
    
    order by 3 desc
""")

In [12]:
conditionSummaryDf.limit(50).toPandas()

,id,conditionCode,count
0,J96.01,Acute respiratory failure with hypoxia,86893
1,I10,Essential (primary) hypertension,83475
2,J12.89,Other viral pneumonia,75208
3,E78.5,"Hyperlipidemia, unspecified",54490
4,Z79.899,Other long term (current) drug therapy,47734
5,N17.9,"Acute kidney failure, unspecified",44292
6,R09.02,Hypoxemia,42611
7,E11.9,Type 2 diabetes mellitus without complications,42370
8,J18.9,"Pneumonia, unspecified organism",38032
9,J12.82,None,36245


In [13]:
conditionSummaryDf.toPandas().to_excel("test4.xlsx")

In [12]:
# spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_HospitalCondition""").show() 

++
||
++
++



In [13]:
# procedureDf.write.saveAsTable(f"""{userName}.{algorithmName}_HospitalProcedure""")

In [14]:
# procedureDf.count()

18084

In [11]:
conditionTransposeDf = spark.sql(f"""


    SELECT distinct 
        h.tenant
        ,h.personId
        ,h.encounterId
        ,c.effectiveDate
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('ABDOMINAL_PAIN_CLIN'), '{defaultcontext}' ),1,0)) as has_ABDOMINAL_PAIN_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('ACUTE_RESPIRATORY_FAILURE_CLIN'), '{defaultcontext}' ),1,0)) as has_ACUTE_RESPIRATORY_FAILURE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('ADVANCED_ILLNESS_CLIN'), '{defaultcontext}' ),1,0)) as has_ADVANCED_ILLNESS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('ALCOHOL_ABUSE_OR_DEPENDENCE_CLIN'), '{defaultcontext}' ),1,0)) as has_ALCOHOL_ABUSE_OR_DEPENDENCE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('ANEMIA_CLIN'), '{defaultcontext}' ),1,0)) as has_ANEMIA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('ANXIETY_CLIN'), '{defaultcontext}' ),1,0)) as has_ANXIETY_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('AORTIC_STENOSIS_CLIN'), '{defaultcontext}' ),1,0)) as has_AORTIC_STENOSIS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('ARRHYTHMIA_CLIN'), '{defaultcontext}' ),1,0)) as has_ARRHYTHMIA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('ASTHMA_CLIN'), '{defaultcontext}' ),1,0)) as has_ASTHMA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('ATHEROSCLEROSIS_CLIN'), '{defaultcontext}' ),1,0)) as has_ATHEROSCLEROSIS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('ATRIAL_FIBRILLATION_CLIN'), '{defaultcontext}' ),1,0)) as has_ATRIAL_FIBRILLATION_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('ATRIAL_FLUTTER_CLIN'), '{defaultcontext}' ),1,0)) as has_ATRIAL_FLUTTER_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('BACTERIAL_INFECTION_OTHER_CLIN'), '{defaultcontext}' ),1,0)) as has_BACTERIAL_INFECTION_OTHER_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('BED_BOUND_CLIN'), '{defaultcontext}' ),1,0)) as has_BED_BOUND_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('BENIGN_PROSTATIC_HYPERPLASIA_CLIN'), '{defaultcontext}' ),1,0)) as has_BENIGN_PROSTATIC_HYPERPLASIA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('BIPOLAR_DISORDER_CLIN'), '{defaultcontext}' ),1,0)) as has_BIPOLAR_DISORDER_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('BMI_ABOVE_NORMAL_ADULT_CLIN'), '{defaultcontext}' ),1,0)) as has_BMI_ABOVE_NORMAL_ADULT_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('BMI_ABOVE_NORMAL_SENIOR_ADULT_CLIN'), '{defaultcontext}' ),1,0)) as has_BMI_ABOVE_NORMAL_SENIOR_ADULT_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('BMI_BELOW_NORMAL_ADULT_CLIN'), '{defaultcontext}' ),1,0)) as has_BMI_BELOW_NORMAL_ADULT_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('BMI_BELOW_NORMAL_SENIOR_ADULT_CLIN'), '{defaultcontext}' ),1,0)) as has_BMI_BELOW_NORMAL_SENIOR_ADULT_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('BODY_MASS_INDEX_BMI_GREATER_THAN_OR_EQUAL_TO_30_AND_LESS_THAN_35_CLIN'), '{defaultcontext}' ),1,0)) as has_BODY_MASS_INDEX_BMI_GREATER_THAN_OR_EQUAL_TO_30_AND_LESS_THAN_35_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('BODY_MASS_INDEX_BMI_GREATER_THAN_OR_EQUAL_TO_35_AND_LESS_THAN_40_CLIN'), '{defaultcontext}' ),1,0)) as has_BODY_MASS_INDEX_BMI_GREATER_THAN_OR_EQUAL_TO_35_AND_LESS_THAN_40_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('BODY_MASS_INDEX_BMI_GREATER_THAN_OR_EQUAL_TO_40_CLIN'), '{defaultcontext}' ),1,0)) as has_BODY_MASS_INDEX_BMI_GREATER_THAN_OR_EQUAL_TO_40_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('BRADYARRHYTHMIA_CLIN'), '{defaultcontext}' ),1,0)) as has_BRADYARRHYTHMIA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CARDIOMYOPATHY_CLIN'), '{defaultcontext}' ),1,0)) as has_CARDIOMYOPATHY_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CARDIOVASCULAR_DEVICES_CLIN'), '{defaultcontext}' ),1,0)) as has_CARDIOVASCULAR_DEVICES_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CEREBROVASCULAR_ACCIDENT_CLIN'), '{defaultcontext}' ),1,0)) as has_CEREBROVASCULAR_ACCIDENT_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CESAREAN_DELIVERY_CLIN'), '{defaultcontext}' ),1,0)) as has_CESAREAN_DELIVERY_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CHEST_PAIN_CLIN'), '{defaultcontext}' ),1,0)) as has_CHEST_PAIN_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CHEST_XRAY_CLIN'), '{defaultcontext}' ),1,0)) as has_CHEST_XRAY_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CHRONIC_AIRWAY_OBSTRUCTION_CLIN'), '{defaultcontext}' ),1,0)) as has_CHRONIC_AIRWAY_OBSTRUCTION_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CHRONIC_KIDNEY_DISEASE_CLIN'), '{defaultcontext}' ),1,0)) as has_CHRONIC_KIDNEY_DISEASE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CHRONIC_KIDNEY_DISEASE_STAGE_3_CLIN'), '{defaultcontext}' ),1,0)) as has_CHRONIC_KIDNEY_DISEASE_STAGE_3_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CHRONIC_KIDNEY_DISEASE_STAGE_4_CLIN'), '{defaultcontext}' ),1,0)) as has_CHRONIC_KIDNEY_DISEASE_STAGE_4_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CHRONIC_KIDNEY_DISEASE_STAGE_5_CLIN'), '{defaultcontext}' ),1,0)) as has_CHRONIC_KIDNEY_DISEASE_STAGE_5_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CHRONIC_LIVER_DISEASE_AND_CIRRHOSIS_CLIN'), '{defaultcontext}' ),1,0)) as has_CHRONIC_LIVER_DISEASE_AND_CIRRHOSIS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CHRONIC_OBSTRUCTIVE_PULMONARY_DISEASE_CLIN'), '{defaultcontext}' ),1,0)) as has_CHRONIC_OBSTRUCTIVE_PULMONARY_DISEASE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CHRONIC_PAIN_CLIN'), '{defaultcontext}' ),1,0)) as has_CHRONIC_PAIN_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CHRONIC_RESPIRATORY_FAILURE_CLIN'), '{defaultcontext}' ),1,0)) as has_CHRONIC_RESPIRATORY_FAILURE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CKD_STAGE_3_4_OR_5_CLIN'), '{defaultcontext}' ),1,0)) as has_CKD_STAGE_3_4_OR_5_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('COGNITIVE_IMPAIRMENT_CLIN'), '{defaultcontext}' ),1,0)) as has_COGNITIVE_IMPAIRMENT_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CONDUCTION_DISORDER_CLIN'), '{defaultcontext}' ),1,0)) as has_CONDUCTION_DISORDER_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CONGESTIVE_HEART_FAILURE_CLIN'), '{defaultcontext}' ),1,0)) as has_CONGESTIVE_HEART_FAILURE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CONSTIPATION_CLIN'), '{defaultcontext}' ),1,0)) as has_CONSTIPATION_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CORONARY_ARTERY_DISEASE_CLIN'), '{defaultcontext}' ),1,0)) as has_CORONARY_ARTERY_DISEASE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CORONAVIRUS_COVID_19_EXPOSURE_TO_CLIN'), '{defaultcontext}' ),1,0)) as has_CORONAVIRUS_COVID_19_EXPOSURE_TO_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CORTICOSTEROID_ADVERSE_REACTION_CLIN'), '{defaultcontext}' ),1,0)) as has_CORTICOSTEROID_ADVERSE_REACTION_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CORTICOSTEROID_LONG_TERM_USE_CLIN'), '{defaultcontext}' ),1,0)) as has_CORTICOSTEROID_LONG_TERM_USE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('COUGH_CLIN'), '{defaultcontext}' ),1,0)) as has_COUGH_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CURRENT_PREGNANCY_NOT_DELIVERED_CLIN'), '{defaultcontext}' ),1,0)) as has_CURRENT_PREGNANCY_NOT_DELIVERED_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('DEMENTIA_CLIN'), '{defaultcontext}' ),1,0)) as has_DEMENTIA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('DEVICE_CLIN'), '{defaultcontext}' ),1,0)) as has_DEVICE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('DIABETES_MELLITUS_TYPE_2_CLIN'), '{defaultcontext}' ),1,0)) as has_DIABETES_MELLITUS_TYPE_2_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('DIABETIC_KETOACIDOSIS_CLIN'), '{defaultcontext}' ),1,0)) as has_DIABETIC_KETOACIDOSIS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('DIABETIC_NEPHROPATHY_CLIN'), '{defaultcontext}' ),1,0)) as has_DIABETIC_NEPHROPATHY_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('DIAPHRAGM_AND_ABDOMINAL_WALL_CLIN'), '{defaultcontext}' ),1,0)) as has_DIAPHRAGM_AND_ABDOMINAL_WALL_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('DIARRHEA_CLIN'), '{defaultcontext}' ),1,0)) as has_DIARRHEA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('DIZZINESS_CLIN'), '{defaultcontext}' ),1,0)) as has_DIZZINESS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('DYSRHYTHMIA_CLIN'), '{defaultcontext}' ),1,0)) as has_DYSRHYTHMIA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('ECG_CLIN'), '{defaultcontext}' ),1,0)) as has_ECG_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('EMPHYSEMA_CLIN'), '{defaultcontext}' ),1,0)) as has_EMPHYSEMA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('ENCEPHALOPATHY_CLIN'), '{defaultcontext}' ),1,0)) as has_ENCEPHALOPATHY_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('ENCEPHALOPATHY_UNSPECIFIED_CLIN'), '{defaultcontext}' ),1,0)) as has_ENCEPHALOPATHY_UNSPECIFIED_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('END_STAGE_RENAL_DISEASE_CLIN'), '{defaultcontext}' ),1,0)) as has_END_STAGE_RENAL_DISEASE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('ENDOCARDIUM_DISEASE_CLIN'), '{defaultcontext}' ),1,0)) as has_ENDOCARDIUM_DISEASE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('ENDOCRINE_DISORDER_CLIN'), '{defaultcontext}' ),1,0)) as has_ENDOCRINE_DISORDER_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('FALL_CLIN'), '{defaultcontext}' ),1,0)) as has_FALL_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('FALL_RISK_CLIN'), '{defaultcontext}' ),1,0)) as has_FALL_RISK_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('FAMILY_HISTORY_OF_DIABETES_MELLITUS_CLIN'), '{defaultcontext}' ),1,0)) as has_FAMILY_HISTORY_OF_DIABETES_MELLITUS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('FATIGUE_CLIN'), '{defaultcontext}' ),1,0)) as has_FATIGUE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('FEVER_CLIN'), '{defaultcontext}' ),1,0)) as has_FEVER_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('FLUID_OVERLOAD_CLIN'), '{defaultcontext}' ),1,0)) as has_FLUID_OVERLOAD_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('GASTROENTERITIS_CLIN'), '{defaultcontext}' ),1,0)) as has_GASTROENTERITIS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('GASTROINTESTINAL_DEVICES_CLIN'), '{defaultcontext}' ),1,0)) as has_GASTROINTESTINAL_DEVICES_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('GENERALIZED_ANXIETY_DISORDER_CLIN'), '{defaultcontext}' ),1,0)) as has_GENERALIZED_ANXIETY_DISORDER_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('HEADACHE_CLIN'), '{defaultcontext}' ),1,0)) as has_HEADACHE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('HEART_FAILURE_CLIN'), '{defaultcontext}' ),1,0)) as has_HEART_FAILURE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('HEMATOCHEZIA_CLIN'), '{defaultcontext}' ),1,0)) as has_HEMATOCHEZIA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('HOMELESS_CLIN'), '{defaultcontext}' ),1,0)) as has_HOMELESS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('HYPERCHOLESTEROLEMIA_CLIN'), '{defaultcontext}' ),1,0)) as has_HYPERCHOLESTEROLEMIA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('HYPERGLYCEMIA_CLIN'), '{defaultcontext}' ),1,0)) as has_HYPERGLYCEMIA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('HYPERLIPIDEMIA_CLIN'), '{defaultcontext}' ),1,0)) as has_HYPERLIPIDEMIA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('HYPERTENSION_CLIN'), '{defaultcontext}' ),1,0)) as has_HYPERTENSION_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('HYPERTENSIVE_CHRONIC_RENAL_DISEASE_CLIN'), '{defaultcontext}' ),1,0)) as has_HYPERTENSIVE_CHRONIC_RENAL_DISEASE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('HYPOGLYCEMIA_CLIN'), '{defaultcontext}' ),1,0)) as has_HYPOGLYCEMIA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('HYPOTENSION_CLIN'), '{defaultcontext}' ),1,0)) as has_HYPOTENSION_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('HYSTERECTOMY_NO_CERVIX_CLIN'), '{defaultcontext}' ),1,0)) as has_HYSTERECTOMY_NO_CERVIX_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('INSOMNIA_CLIN'), '{defaultcontext}' ),1,0)) as has_INSOMNIA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('IVD_CLIN'), '{defaultcontext}' ),1,0)) as has_IVD_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('LIPID_METABOLISM_CLIN'), '{defaultcontext}' ),1,0)) as has_LIPID_METABOLISM_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('LIVE_BIRTH_CLIN'), '{defaultcontext}' ),1,0)) as has_LIVE_BIRTH_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('LIVER_CIRRHOSIS_CLIN'), '{defaultcontext}' ),1,0)) as has_LIVER_CIRRHOSIS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('LIVER_DISEASE_CLIN'), '{defaultcontext}' ),1,0)) as has_LIVER_DISEASE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('LONG_TERM_OXYGEN_THERAPY_CLIN'), '{defaultcontext}' ),1,0)) as has_LONG_TERM_OXYGEN_THERAPY_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('LOW_BACK_PAIN_CLIN'), '{defaultcontext}' ),1,0)) as has_LOW_BACK_PAIN_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('MAJOR_DEPRESSIVE_DISORDER_CLIN'), '{defaultcontext}' ),1,0)) as has_MAJOR_DEPRESSIVE_DISORDER_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('MORBID_OBESITY_CLIN'), '{defaultcontext}' ),1,0)) as has_MORBID_OBESITY_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('MOVEMENT_DISEASE_CLIN'), '{defaultcontext}' ),1,0)) as has_MOVEMENT_DISEASE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('MYOCARDIAL_INFARCTION_CLIN'), '{defaultcontext}' ),1,0)) as has_MYOCARDIAL_INFARCTION_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('NAUSEA_CLIN'), '{defaultcontext}' ),1,0)) as has_NAUSEA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('NON_AMBULATORY_CLIN'), '{defaultcontext}' ),1,0)) as has_NON_AMBULATORY_CLIN
--,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('NOSE_BLEED_CLIN'), '{defaultcontext1}' ),1,0)) as has_NOSE_BLEED_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('OBESITY_CLIN'), '{defaultcontext}' ),1,0)) as has_OBESITY_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('OBSTRUCTIVE_SLEEP_APNEA_CLIN'), '{defaultcontext}' ),1,0)) as has_OBSTRUCTIVE_SLEEP_APNEA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('OSTEOPOROSIS_CLIN'), '{defaultcontext}' ),1,0)) as has_OSTEOPOROSIS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('OTHER_GASTROINTESTINAL_CLIN'), '{defaultcontext}' ),1,0)) as has_OTHER_GASTROINTESTINAL_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('PLEURISY_CLIN'), '{defaultcontext}' ),1,0)) as has_PLEURISY_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('PNEUMONIA_CLIN'), '{defaultcontext}' ),1,0)) as has_PNEUMONIA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('PNEUMONIA_NON_VIRAL_CLIN'), '{defaultcontext}' ),1,0)) as has_PNEUMONIA_NON_VIRAL_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('PREDIABETES_CLIN'), '{defaultcontext}' ),1,0)) as has_PREDIABETES_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('PREGNANCY_CLIN'), '{defaultcontext}' ),1,0)) as has_PREGNANCY_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('PSYCHOTIC_DISORDER_CLIN'), '{defaultcontext}' ),1,0)) as has_PSYCHOTIC_DISORDER_CLIN
--,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('PULMONARY_BLEED_CLIN'), '{defaultcontext1}' ),1,0)) as has_PULMONARY_BLEED_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('PULMONARY_EMBOLISM_CLIN'), '{defaultcontext}' ),1,0)) as has_PULMONARY_EMBOLISM_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('PULMONARY_HYPERTENSION_CLIN'), '{defaultcontext}' ),1,0)) as has_PULMONARY_HYPERTENSION_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('RENAL_DIALYSIS_CLIN'), '{defaultcontext}' ),1,0)) as has_RENAL_DIALYSIS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('RENAL_FAILURE_ACUTE_CLIN'), '{defaultcontext}' ),1,0)) as has_RENAL_FAILURE_ACUTE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('RENAL_FAILURE_CHRONIC_CLIN'), '{defaultcontext}' ),1,0)) as has_RENAL_FAILURE_CHRONIC_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('RESPIRATORY_DEVICES_CLIN'), '{defaultcontext}' ),1,0)) as has_RESPIRATORY_DEVICES_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('RESPIRATORY_DISEASE_SPECIFIED_OTHER_CLIN'), '{defaultcontext}' ),1,0)) as has_RESPIRATORY_DISEASE_SPECIFIED_OTHER_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('RESPIRATORY_DISTRESS_CLIN'), '{defaultcontext}' ),1,0)) as has_RESPIRATORY_DISTRESS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('RHABDOMYOLYSIS_CLIN'), '{defaultcontext}' ),1,0)) as has_RHABDOMYOLYSIS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('RHEUMATOID_ARTHRITIS_CLIN'), '{defaultcontext}' ),1,0)) as has_RHEUMATOID_ARTHRITIS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('SCHIZOPHRENIA_CLIN'), '{defaultcontext}' ),1,0)) as has_SCHIZOPHRENIA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('SCHIZOPHRENIA_OR_PSYCHOTIC_DISORDER_CLIN'), '{defaultcontext}' ),1,0)) as has_SCHIZOPHRENIA_OR_PSYCHOTIC_DISORDER_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('SEIZURE_CLIN'), '{defaultcontext}' ),1,0)) as has_SEIZURE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('SHORTNESS_OF_BREATH_CLIN'), '{defaultcontext}' ),1,0)) as has_SHORTNESS_OF_BREATH_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('SMOKELESS_TOBACCO_USER_CLIN'), '{defaultcontext}' ),1,0)) as has_SMOKELESS_TOBACCO_USER_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('SUBSTANCE_ABUSE_CLIN'), '{defaultcontext}' ),1,0)) as has_SUBSTANCE_ABUSE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('SUICIDAL_IDEATION_CLIN'), '{defaultcontext}' ),1,0)) as has_SUICIDAL_IDEATION_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('TOBACCO_USER_CLIN'), '{defaultcontext}' ),1,0)) as has_TOBACCO_USER_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('UPPER_RESPIRATORY_INFECTION_ACUTE_CLIN'), '{defaultcontext}' ),1,0)) as has_UPPER_RESPIRATORY_INFECTION_ACUTE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('URINARY_INCONTINENCE_CLIN'), '{defaultcontext}' ),1,0)) as has_URINARY_INCONTINENCE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('URINARY_TRACT_INFECTION_CLIN'), '{defaultcontext}' ),1,0)) as has_URINARY_TRACT_INFECTION_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('VAGINAL_DELIVERY_CLIN'), '{defaultcontext}' ),1,0)) as has_VAGINAL_DELIVERY_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('VERTIGO_CLIN'), '{defaultcontext}' ),1,0)) as has_VERTIGO_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('VITAMIN_D_DEFICIENCY_CLIN'), '{defaultcontext}' ),1,0)) as has_VITAMIN_D_DEFICIENCY_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('VOMITING_CLIN'), '{defaultcontext}' ),1,0)) as has_VOMITING_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('WHEELCHAIR_BOUND_CLIN'), '{defaultcontext}' ),1,0)) as has_WHEELCHAIR_BOUND_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_101_CORONARY_ATHEROSCLEROSIS_HEART_DISEASE_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_101_CORONARY_ATHEROSCLEROSIS_HEART_DISEASE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_103_PULMONARY_HEART_DISEASE_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_103_PULMONARY_HEART_DISEASE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_104_OTHER_AND_ILL_DEFINED_HEART_DISEASE_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_104_OTHER_AND_ILL_DEFINED_HEART_DISEASE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_105_CONDUCTION_DISORDERS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_105_CONDUCTION_DISORDERS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_106_CARDIAC_DYSRHYTHMIAS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_106_CARDIAC_DYSRHYTHMIAS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_108_CONGESTIVE_HEART_FAILURE_NONHYPERTENSIVE_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_108_CONGESTIVE_HEART_FAILURE_NONHYPERTENSIVE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_109_ACUTE_CEREBROVASCULAR_DISEASE_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_109_ACUTE_CEREBROVASCULAR_DISEASE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_11_CANCER_OF_HEAD_AND_NECK_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_11_CANCER_OF_HEAD_AND_NECK_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_112_TRANSIENT_CEREBRAL_ISCHEMIA_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_112_TRANSIENT_CEREBRAL_ISCHEMIA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_113_LATE_EFFECTS_OF_CEREBROVASCULAR_DISEASE_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_113_LATE_EFFECTS_OF_CEREBROVASCULAR_DISEASE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_114_PERIPHERAL_AND_VISCERAL_ATHEROSCLEROSIS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_114_PERIPHERAL_AND_VISCERAL_ATHEROSCLEROSIS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_115_AORTIC_PERIPHERAL_VISCERAL_ARTERY_ANEURYSMS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_115_AORTIC_PERIPHERAL_VISCERAL_ARTERY_ANEURYSMS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_117_OTHER_CIRCULATORY_DISEASE_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_117_OTHER_CIRCULATORY_DISEASE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_12_CANCER_OF_ESOPHAGUS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_12_CANCER_OF_ESOPHAGUS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_121_OTHER_DISEASES_VEINS_LYMPHATICS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_121_OTHER_DISEASES_VEINS_LYMPHATICS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_122_PNEUMONIA_EXCEPT_TB_OR_STD_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_122_PNEUMONIA_EXCEPT_TB_OR_STD_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_127_COPD_BRONCHIECTASIS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_127_COPD_BRONCHIECTASIS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_128_ASTHMA_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_128_ASTHMA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_129_ASPIRATION_PNEUMONITIS_FOOD_VOMITUS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_129_ASPIRATION_PNEUMONITIS_FOOD_VOMITUS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_13_CANCER_OF_STOMACH_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_13_CANCER_OF_STOMACH_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_139_GASTRODUODENAL_ULCER_EXCEPT_HEMORRHAGE_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_139_GASTRODUODENAL_ULCER_EXCEPT_HEMORRHAGE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_14_CANCER_OF_COLON_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_14_CANCER_OF_COLON_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_15_CANCER_OF_RECTUM_AND_ANUS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_15_CANCER_OF_RECTUM_AND_ANUS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_151_OTHER_LIVER_DISEASES_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_151_OTHER_LIVER_DISEASES_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_156_NEPHRITIS_NEPHROSIS_RENAL_SCLEROSIS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_156_NEPHRITIS_NEPHROSIS_RENAL_SCLEROSIS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_157_ACUTE_UNSPECIFIED_RENAL_FAILURE_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_157_ACUTE_UNSPECIFIED_RENAL_FAILURE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_158_CHRONIC_KIDNEY_DISEASE_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_158_CHRONIC_KIDNEY_DISEASE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_16_CANCER_OF_LIVER_AND_INTRAHEPATIC_BILE_DUCT_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_16_CANCER_OF_LIVER_AND_INTRAHEPATIC_BILE_DUCT_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_17_CANCER_OF_PANCREAS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_17_CANCER_OF_PANCREAS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_18_CANCER_OF_OTHER_GI_ORGANS_PERITONEUM_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_18_CANCER_OF_OTHER_GI_ORGANS_PERITONEUM_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_19_CANCER_OF_BRONCHUS_LUNG_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_19_CANCER_OF_BRONCHUS_LUNG_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_2_SEPTICEMIA_EXECPT_IN_LABOR_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_2_SEPTICEMIA_EXECPT_IN_LABOR_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_20_CANCER_OTHER_RESPIRATORY_AND_INTRATHORACIC_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_20_CANCER_OTHER_RESPIRATORY_AND_INTRATHORACIC_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_207_PATHOLOGICAL_FRACTURE_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_207_PATHOLOGICAL_FRACTURE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_21_CANCER_OF_BONE_AND_CONNECTIVE_TISSUE_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_21_CANCER_OF_BONE_AND_CONNECTIVE_TISSUE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_210_LUPUS_AND_CONNECTIVE_TISSUE_DISORDERS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_210_LUPUS_AND_CONNECTIVE_TISSUE_DISORDERS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_211_OTHER_CONNECTIVE_TISSUE_DISEASE_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_211_OTHER_CONNECTIVE_TISSUE_DISEASE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_213_CARDIAC_CIRCULATORY_CONGENITAL_ANOMALIES_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_213_CARDIAC_CIRCULATORY_CONGENITAL_ANOMALIES_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_22_MELANOMAS_OF_SKIN_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_22_MELANOMAS_OF_SKIN_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_226_FRACTURE_NECK_OF_FEMUR_HIP_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_226_FRACTURE_NECK_OF_FEMUR_HIP_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_23_OTHER_NON_EPITHELIAL_CANCER_OF_SKIN_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_23_OTHER_NON_EPITHELIAL_CANCER_OF_SKIN_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_230_FRACTURE_OF_LOWER_LIMB_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_230_FRACTURE_OF_LOWER_LIMB_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_237_DEVICE_COMPLICATION_IMPLANT_OR_GRAFT_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_237_DEVICE_COMPLICATION_IMPLANT_OR_GRAFT_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_238_COMPLICATIONS_SURGICAL_PROC_OR_MEDICAL_CARE_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_238_COMPLICATIONS_SURGICAL_PROC_OR_MEDICAL_CARE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_24_CANCER_OF_BREAST_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_24_CANCER_OF_BREAST_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_242_POISONING_BY_OTHER_MEDICATIONS_AND_DRUGS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_242_POISONING_BY_OTHER_MEDICATIONS_AND_DRUGS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_244_OTHER_INJURIES_AND_CONDITIONS_DUE_TO_EXTERNAL_CAUSES_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_244_OTHER_INJURIES_AND_CONDITIONS_DUE_TO_EXTERNAL_CAUSES_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_245_SYNCOPE_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_245_SYNCOPE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_25_CANCER_OF_UTERUS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_25_CANCER_OF_UTERUS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_253_ALLERGIC_REACTIONS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_253_ALLERGIC_REACTIONS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_258_OTHER_SCREENING_FOR_SUSPECTED_CONDITIONS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_258_OTHER_SCREENING_FOR_SUSPECTED_CONDITIONS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_259_RESIDUAL_CODES_UNCLASSIFIED_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_259_RESIDUAL_CODES_UNCLASSIFIED_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_26_CANCER_OF_CERVIX_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_26_CANCER_OF_CERVIX_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_27_CANCER_OF_OVARY_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_27_CANCER_OF_OVARY_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_28_CANCER_OTHER_FEMALE_GENITAL_ORGANS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_28_CANCER_OTHER_FEMALE_GENITAL_ORGANS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_29_CANCER_OF_PROSTATE_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_29_CANCER_OF_PROSTATE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_30_CANCER_OF_TESTIS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_30_CANCER_OF_TESTIS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_31_CANCER_OTHER_MALE_GENITAL_ORGANS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_31_CANCER_OTHER_MALE_GENITAL_ORGANS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_32_CANCER_OF_BLADDER_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_32_CANCER_OF_BLADDER_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_33_CANCER_KIDNEY_AND_RENAL_PELVIS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_33_CANCER_KIDNEY_AND_RENAL_PELVIS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_34_CANCER_OTHER_URINARY_ORGANS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_34_CANCER_OTHER_URINARY_ORGANS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_35_CANCER_BRAIN_NERVOUS_SYSTEM_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_35_CANCER_BRAIN_NERVOUS_SYSTEM_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_36_CANCER_OF_THYROID_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_36_CANCER_OF_THYROID_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_37_HODGKINS_DISEASE_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_37_HODGKINS_DISEASE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_38_NON_HODGKINS_LYMPHOMA_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_38_NON_HODGKINS_LYMPHOMA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_39_LEUKEMIAS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_39_LEUKEMIAS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_40_MULTIPLE_MYELOMA_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_40_MULTIPLE_MYELOMA_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_41_CANCER_OTHER_AND_UNSPECIFIED_PRIMARY_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_41_CANCER_OTHER_AND_UNSPECIFIED_PRIMARY_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_42_SECONDARY_MALIGNANCIES_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_42_SECONDARY_MALIGNANCIES_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_43_MALIGNANT_NEOPLASM_WITHOUT_SPECIFICATION_OF_SITE_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_43_MALIGNANT_NEOPLASM_WITHOUT_SPECIFICATION_OF_SITE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_49_DIABETES_MELLITUS_WITHOUT_COMPLICATION_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_49_DIABETES_MELLITUS_WITHOUT_COMPLICATION_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_5_HIV_INFECTION_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_5_HIV_INFECTION_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_50_DIABETES_MELLITUS_WITH_COMPLICATIONS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_50_DIABETES_MELLITUS_WITH_COMPLICATIONS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_52_NUTRITIONAL_DEFICIENCIES_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_52_NUTRITIONAL_DEFICIENCIES_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_55_FLUID_AND_ELECTROLYTE_DISORDERS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_55_FLUID_AND_ELECTROLYTE_DISORDERS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_58_NUTRITIONAL_ENDOCRINE_METABOLIC_DISORDERS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_58_NUTRITIONAL_ENDOCRINE_METABOLIC_DISORDERS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_6_HEPATITIS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_6_HEPATITIS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_62_COAGULATION_HEMORRHAGIC_DISORDERS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_62_COAGULATION_HEMORRHAGIC_DISORDERS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_64_OTHER_HEMATOLOGIC_CONDITIONS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_64_OTHER_HEMATOLOGIC_CONDITIONS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_659_SCHIZOPHRENIA_AND_OTHER_PSYCHOTIC_DISORDERS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_659_SCHIZOPHRENIA_AND_OTHER_PSYCHOTIC_DISORDERS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_660_ALCOHOL_RELATED_DISORDERS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_660_ALCOHOL_RELATED_DISORDERS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_663_SCREEN_AND_HISTORY_MENTAL_HEALTH_AND_SUB_ABUSE_CODES_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_663_SCREEN_AND_HISTORY_MENTAL_HEALTH_AND_SUB_ABUSE_CODES_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_7_VIRAL_INFECTION_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_7_VIRAL_INFECTION_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_77_ENCEPHALITIS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_77_ENCEPHALITIS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_81_HEREDITARY_DEGENERATIVE_NERVOUS_SYSTEM_CONDITIONS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_81_HEREDITARY_DEGENERATIVE_NERVOUS_SYSTEM_CONDITIONS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_95_OTHER_NERVOUS_SYSTEM_DISORDERS_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_95_OTHER_NERVOUS_SYSTEM_DISORDERS_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_97_PERI_ENDO_MYOCARDITIS_CARDIOMYOPATHY_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_97_PERI_ENDO_MYOCARDITIS_CARDIOMYOPATHY_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CCS_CATEGORY_99_HYPERTENSION_AND_SECONDARY_HYPERTENSION_CLIN'), '{defaultcontext1}' ),1,0)) as has_CCS_CATEGORY_99_HYPERTENSION_AND_SECONDARY_HYPERTENSION_CLIN
--,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('LIVE_BIRTH_CLIN'), '{defaultcontext1}' ),1,0)) as has_LIVE_BIRTH_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('STILLBIRTH_CLIN'), '{defaultcontext1}' ),1,0)) as has_STILLBIRTH_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('TERMINATION_OF_PREGNANCY_CLIN'), '{defaultcontext1}' ),1,0)) as has_TERMINATION_OF_PREGNANCY_CLIN
--,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('VAGINAL_DELIVERY_CLIN'), '{defaultcontext1}' ),1,0)) as has_VAGINAL_DELIVERY_CLIN
--,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CESAREAN_DELIVERY_CLIN'), '{defaultcontext1}' ),1,0)) as has_CESAREAN_DELIVERY_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('HOSPICE_CARE_CLIN'), '{defaultcontext1}' ),1,0)) as has_HOSPICE_CARE_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('SEPSIS_CLIN'), '{defaultcontext1}' ),1,0)) as has_SEPSIS_CLIN
--,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('COMPLICATION_OR_COMORBIDITY_CLIN'), '{defaultcontext1}' ),1,0)) as has_COMPLICATION_OR_COMORBIDITY_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('COMPLICATIONS_OF_PREGNANCY_CHILDBIRTH_AND_THE_PUERPERIUM_CLIN'), '{defaultcontext1}' ),1,0)) as has_COMPLICATIONS_OF_PREGNANCY_CHILDBIRTH_AND_THE_PUERPERIUM_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CANCER_CLIN'), '{defaultcontext1}' ),1,0)) as has_CANCER_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('DOWN_SYNDROME_TRISOMY_21_CLIN'), '{defaultcontext1}' ),1,0)) as has_DOWN_SYNDROME_TRISOMY_21_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('HEART_TRANSPLANT_CLIN'), '{defaultcontext1}' ),1,0)) as has_HEART_TRANSPLANT_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('LIVER_TRANSPLANT_CLIN'), '{defaultcontext1}' ),1,0)) as has_LIVER_TRANSPLANT_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('LUNG_TRANSPLANT_CLIN'), '{defaultcontext1}' ),1,0)) as has_LUNG_TRANSPLANT_CLIN
,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('RENAL_TRANSPLANT_CLIN'), '{defaultcontext1}' ),1,0)) as has_RENAL_TRANSPLANT_CLIN
  
        FROM {userName}.{algorithmName}_Hospital h  

       

        JOIN {db}.condition c
            ON  h.personId = c.personId
                and h.tenant = c.tenant
                and h.encounterId = c.encounterId
                AND  NOT ( HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CORONAVIRUS_COVID_19_DISEASE_CLIN',
                                        'CORONAVIRUS_COVID_19_POSITIVE_CLIN'), '{defaultcontext}' ) )
                AND NOT HAS_ANY_CONCEPT_IN_CONTEXT(c.status, array('ENTERED_IN_ERROR_PROBSTAT', 'ERROR_ENTRY_DELETED_QUAL', 
                                                       'CANCELED_PROBSTAT', 'CANCELED_QUAL', 'DEFERRED_QUAL',
                                                       'UNAUTHORIZED_QUAL', 'VOIDED_QUAL', 'REJECTED_QUAL'), '5E259FD575B54D4982D32D4E92DCA831')  
                
                            
 GROUP BY 1,2,3,4    
                       
 ORDER BY 1,2,3,4  
   
""")

In [12]:
conditionTransposeDf.limit(25).toPandas()

,tenant,personId,encounterId,effectiveDate,has_ABDOMINAL_PAIN_CLIN,has_ACUTE_RESPIRATORY_FAILURE_CLIN,has_ADVANCED_ILLNESS_CLIN,has_ALCOHOL_ABUSE_OR_DEPENDENCE_CLIN,has_ANEMIA_CLIN,has_ANXIETY_CLIN,has_AORTIC_STENOSIS_CLIN,has_ARRHYTHMIA_CLIN,has_ASTHMA_CLIN,has_ATHEROSCLEROSIS_CLIN,has_ATRIAL_FIBRILLATION_CLIN,has_ATRIAL_FLUTTER_CLIN,has_BACTERIAL_INFECTION_OTHER_CLIN,...,has_CCS_CATEGORY_7_VIRAL_INFECTION_CLIN,has_CCS_CATEGORY_77_ENCEPHALITIS_CLIN,has_CCS_CATEGORY_81_HEREDITARY_DEGENERATIVE_NERVOUS_SYSTEM_CONDITIONS_CLIN,has_CCS_CATEGORY_95_OTHER_NERVOUS_SYSTEM_DISORDERS_CLIN,has_CCS_CATEGORY_97_PERI_ENDO_MYOCARDITIS_CARDIOMYOPATHY_CLIN,has_CCS_CATEGORY_99_HYPERTENSION_AND_SECONDARY_HYPERTENSION_CLIN,has_STILLBIRTH_CLIN,has_TERMINATION_OF_PREGNANCY_CLIN,has_HOSPICE_CARE_CLIN,has_SEPSIS_CLIN,has_COMPLICATIONS_OF_PREGNANCY_CHILDBIRTH_AND_THE_PUERPERIUM_CLIN,has_CANCER_CLIN,has_DOWN_SYNDROME_TRISOMY_21_CLIN,has_HEART_TRANSPLANT_CLIN,has_LIVER_TRANSPLANT_CLIN,has_LUNG_TRANSPLANT_CLIN,has_RENAL_TRANSPLANT_CLIN
0,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-11T04:33:00+00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-11T04:34:00+00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,002756bd-6290-4982-9c69-f879158503ab,32d82cb7-69b8-4d9d-ab2d-b97e14c59927,,0,1,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,1,00368fc1-dd5b-4aa5-82f8-2a6c823364d3,2a36bc40-1dbe-46dd-b774-83d269d1992e,,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,00368fc1-dd5b-4aa5-82f8-2a6c823364d3,2a36bc40-1dbe-46dd-b774-83d269d1992e,2020-10-25T07:05:00+00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,00368fc1-dd5b-4aa5-82f8-2a6c823364d3,2a36bc40-1dbe-46dd-b774-83d269d1992e,2020-10-25T07:07:00+00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,1,004f0a70-f538-4e7a-b426-3b7306a59a80,ee12825a-d8a3-4e64-8de6-cd5228c8c16f,,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
8,1,007d5eca-d606-40d2-9031-af55a7b01dab,86b9bb4f-beb4-433a-aca3-6f32efe5d940,,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1,007d5eca-d606-40d2-9031-af55a7b01dab,86b9bb4f-beb4-433a-aca3-6f32efe5d940,2020-10-03T21:19:00+00:00,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
# Drop existing table, if there was one 
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_SummaryPatientsConditions""").show() 

++
||
++
++



In [14]:
conditionTransposeDf.write.saveAsTable(f"""{userName}.{algorithmName}_SummaryPatientsConditions """)

In [15]:
spark.sql(f"""select count(distinct personId) from {userName}.{algorithmName}_SummaryPatientsConditions """).show()

+------------------------+
|count(DISTINCT personId)|
+------------------------+
|                  176650|
+------------------------+



In [16]:
spark.sql(f"""select count(distinct personId) from {userName}.{algorithmName}_Hospital """).show()

+------------------------+
|count(DISTINCT personId)|
+------------------------+
|                  177699|
+------------------------+

